In [1]:
import pyodbc 
import pandas as pd
import datetime
import numpy as np

In [2]:
'''Connect to post result SQL server'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=dsa-ln-TQA1V;DATABASE=qai;Trusted_Connection=yes')   
cursor = cnxn.cursor() 

In [3]:
datetime.datetime.now()

datetime.datetime(2018, 12, 27, 12, 16, 13, 289382)

## Data acquisition

In [4]:
'''
Broker level estimates
'''

ibdestl_sql="select * from IBDEstL1 where PerType=1 and Period<3 and year(EstDate)>2016 and year(EstDate)<2018"
ibdestl=pd.read_sql(ibdestl_sql,cnxn)

In [7]:
'''
Daily consensus data
'''

ibesusdlyest_sql=\
"select * from IBESUSDLYEST where PerType=1 and Period<3 and Measure=8 and year(EstDate)>2016 and year(EstDate)<2018"
ibesusdlyest=pd.read_sql(ibesusdlyest_sql,cnxn)

In [6]:
'''
Actual reporting data
'''

ibdactl_sql="select * from IBDActL1 where PerType='Q' and year(RptDate)>2016 and year(RptDate)<2018"
ibdactl=pd.read_sql(ibdactl_sql,cnxn)

In [61]:
'''
Dataframe with mean, median, count
mean estimate vs previous consensus
mean revision vs previous consensus
mean revision vs previous estimate
same thing applies to year 1 and 2
'''

## Key utility functions

In [185]:
def key_IBES_metrics(code,date,est_period,revision_period,broker_df,consensus_df):
    '''
    utility function to generate key metrics from broker level data and consensus data
    '''
    broker1=broker_df[(broker_df.Code==code)&(broker_df.Period==1)]
    broker2=broker_df[(broker_df.Code==code)&(broker_df.Period==2)]
    
    consensus1=consensus_df[(consensus_df.Code==code)&(consensus_df.Period==1)]
    consensus2=consensus_df[(consensus_df.Code==code)&(consensus_df.Period==2)]                 
    
    try:
        broker1_est=broker1[(broker1.EstDate>=date)&(broker1.EstDate<=date+datetime.timedelta(est_period))]    
        pre_consensus1=consensus1[(consensus1.Code==code)&(consensus1.EstDate<=date)].sort_values("EstDate").iloc[-1]["Mean"]
        post_consensus1=consensus1[(consensus1.Code==code)&(consensus1.EstDate>date)&\
                                   (consensus1.EstDate<=date+datetime.timedelta(revision_period))].\
                                    sort_values("EstDate").iloc[0]["Mean"]
        broker1_mean_from_consensus=broker1_est["Value_"].mean()/pre_consensus1-1
        broker1_median_from_consensus=broker1_est["Value_"].median()/pre_consensus1-1
        actual_revision1=post_consensus1/pre_consensus1-1
        count1=broker1_est.shape[0]
        
    except IndexError:
        broker1_mean_from_consensus=None
        broker1_median_from_consensus=None
        actual_revision1=None
        count1=None

    try:
        broker2_est=broker2[(broker2.EstDate>=date)&(broker2.EstDate<=date+datetime.timedelta(est_period))]    
        pre_consensus2=consensus2[(consensus2.Code==code)&(consensus2.EstDate<=date)].sort_values("EstDate").iloc[-1]["Mean"]
        post_consensus2=consensus2[(consensus2.Code==code)&(consensus2.EstDate>date)&\
                                   (consensus2.EstDate<=date+datetime.timedelta(revision_period))].\
                                    sort_values("EstDate").iloc[0]["Mean"]
        broker2_mean_from_consensus=broker2_est["Value_"].mean()/pre_consensus2-1
        broker2_median_from_consensus=broker2_est["Value_"].median()/pre_consensus2-1
        actual_revision2=post_consensus2/pre_consensus2-1
        count2=broker2_est.shape[0]
        
    except IndexError:
        broker2_mean_from_consensus=None
        broker2_median_from_consensus=None
        actual_revision2=None
        count2=None        

    
    return (broker1_mean_from_consensus,broker1_median_from_consensus,actual_revision1,count1),\
            (broker2_mean_from_consensus,broker2_median_from_consensus,actual_revision2,count2)


    

In [186]:
alist=list(zip(ibdactl.Code,ibdactl.RptDate,ibdactl.PerDate))

In [199]:
target=alist[130]

In [200]:
target

(290, Timestamp('2017-07-31 00:00:00'), Timestamp('2017-06-30 00:00:00'))

In [201]:
key_IBES_metrics(target[0],target[1],2,20,ibdestl,ibesusdlyest)

((0.1390070421236076, 0.13439711324826287, 0.09893618141955507, 7),
 (0.1096930245992227, 0.13970337372577912, 0.07795797566090856, 7))

In [184]:
ibdestl[(ibdestl.Code==target[0])&(ibdestl.EstDate>=target[1])&(ibdestl.EstDate<=target[1]+datetime.timedelta(2))]

,Code,Measure,Broker,Analyst,PerType,Period,EstDate,PerDate,Currency_,PDFlag,Value_,RevDate,IsoCurrency
118,16,8,54417,511595,1,1,2017-08-02,2017-12-31,,D,1.90,2017-08-02,None
121,16,8,54417,511595,1,2,2017-08-02,2018-12-31,,D,2.36,2017-08-02,None


In [143]:
ibdestl[(ibdestl.Code==target[0])]

,Code,Measure,Broker,Analyst,PerType,Period,EstDate,PerDate,Currency_,PDFlag,Value_,RevDate,IsoCurrency
114,16,8,2394,114449,1,1,2017-02-27,2017-12-31,,D,2.164,2017-03-23,None
115,16,8,2394,114449,1,2,2017-02-27,2018-12-31,,D,2.499,2017-03-23,None
116,16,8,54417,511595,1,1,2017-02-17,2017-12-31,,D,2.210,2017-02-17,None
117,16,8,54417,511595,1,1,2017-05-10,2017-12-31,,D,2.020,2017-07-14,None
118,16,8,54417,511595,1,1,2017-08-02,2017-12-31,,D,1.900,2017-08-02,None
119,16,8,54417,511595,1,1,2017-11-01,2017-12-31,,D,1.960,2017-11-30,None
120,16,8,54417,511595,1,2,2017-05-10,2018-12-31,,D,2.480,2017-07-14,None
121,16,8,54417,511595,1,2,2017-08-02,2018-12-31,,D,2.360,2017-08-02,None
122,16,8,54417,511595,1,2,2017-11-01,2018-12-31,,D,2.390,2017-11-30,None


In [53]:
'''
Pre value
'''
ibesusdlyest[(ibesusdlyest.Code==target[0])&(ibesusdlyest.EstDate<=target[1])&(ibesusdlyest.Period==1)].\
sort_values("EstDate").iloc[-1]["Mean"]

1.0888888835907

In [54]:
'''
Post value
'''
ibesusdlyest[(ibesusdlyest.Code==target[0])&(ibesusdlyest.EstDate>=target[1]+datetime.timedelta(20))&(ibesusdlyest.Period==1)].\
sort_values("EstDate").iloc[0]["Mean"]

1.15422224998474

In [25]:
ibesusdlyest[ibesusdlyest.Code==target[0]]

,Code,Measure,EstDate,PerDate,Period,PerType,NumEst,Median,Mean,StdDev,High,Low,PdFlag,Currency_
7427,849,8,2017-08-11,2017-12-31,1,1,1,-0.74,-0.74,NaN,-0.74,-0.74,D,USD
7428,849,8,2017-08-11,2018-12-31,2,1,1,-0.40,-0.40,NaN,-0.40,-0.40,D,USD
7962,849,8,2017-03-28,2017-12-31,1,1,1,-1.09,-1.09,NaN,-1.09,-1.09,D,USD
7972,849,8,2017-03-29,2017-12-31,1,1,1,-1.45,-1.45,NaN,-1.45,-1.45,D,USD
8126,849,8,2017-05-17,2017-12-31,1,1,1,-1.10,-1.10,NaN,-1.10,-1.10,D,USD
8132,849,8,2017-05-17,2018-12-31,2,1,1,-0.38,-0.38,NaN,-0.38,-0.38,D,USD


In [194]:
ibesusdlyest[(ibesusdlyest.Code==target[0])&(ibesusdlyest.EstDate>=target[1]+datetime.timedelta(20))&(ibesusdlyest.Period==1)].\
sort_values("EstDate")

,Code,Measure,EstDate,PerDate,Period,PerType,NumEst,Median,Mean,StdDev,High,Low,PdFlag,Currency_
7436,919,8,2017-02-28,2017-12-31,1,1,10,6.980,7.001000,0.125738,7.25,6.840,D,USD
7440,919,8,2017-03-01,2017-12-31,1,1,10,6.980,7.026000,0.189455,7.50,6.840,D,USD
7706,919,8,2017-03-21,2017-12-31,1,1,10,6.980,7.029000,0.189763,7.50,6.840,D,USD
7707,919,8,2017-03-24,2017-12-31,1,1,10,7.030,7.040000,0.187202,7.50,6.840,D,USD
7708,919,8,2017-04-10,2017-12-31,1,1,10,7.030,7.035000,0.173734,7.45,6.840,D,USD
7638,919,8,2017-05-04,2017-12-31,1,1,10,7.065,7.080000,0.184451,7.45,6.840,D,USD
7565,919,8,2017-05-05,2017-12-31,1,1,10,7.115,7.108000,0.181647,7.40,6.700,D,USD
7324,919,8,2017-06-23,2017-12-31,1,1,10,7.115,7.121000,0.151324,7.40,6.830,D,USD
7787,919,8,2017-07-18,2017-12-31,1,1,10,7.115,7.125000,0.154434,7.40,6.830,D,USD
7788,919,8,2017-07-19,2017-12-31,1,1,10,7.140,7.138000,0.162056,7.40,6.830,D,USD


In [7]:
'''
Daily consensus data
'''

ibesusdlyest_sql=\
"select * from IBESUSDLYEST where PerType=1 and Period<3 and Measure=8 and year(EstDate)>2016 and year(EstDate)<2018"
ibesusdlyest=pd.read_sql(ibesusdlyest_sql,cnxn)

In [6]:
'''
Actual reporting data
'''

ibdactl_sql="select * from IBDActL1 where PerType='Q' and year(RptDate)>2016 and year(RptDate)<2018"
ibdactl=pd.read_sql(ibdactl_sql,cnxn)

In [9]:
alist=list(zip(ibdactl.Code,ibdactl.RptDate,ibdactl.PerDate))

In [10]:
target=alist[500]

In [11]:
ibdestl[(ibdestl.Code==target[0])&(ibdestl.EstDate>=target[1])&(ibdestl.EstDate<=target[1]+datetime.timedelta(2))]

,Code,Measure,Broker,Analyst,PerType,Period,EstDate,PerDate,Currency_,PDFlag,Value_,RevDate,IsoCurrency


In [99]:
sum_df=pd.DataFrame(index=pd.MultiIndex.from_tuples(alist))

In [40]:
ibesinfo3_sql="select * from IBESInfo3"
ibesinfo3=pd.read_sql(ibesinfo3_sql,cnxn)

In [186]:
ibesinfo3[ibesinfo3["Code"]==330]

,ITicker,Cusip,STicker,Name,Country,Currency_,OneTenth,SecType,ExchCtry,Exchange,Code
1209,AFL,00105510,AFL,AFLAC INC,NA,USD,0,S,NA,A,330
